# Spatial Autocorrelation Model

In [1]:
import pandas as pd
import numpy as np
import csv

#Dependencies: conda install -c conda-forge fiona shapely pyproj rtree
import geopandas as gpd # geopandas.org for more information.
from shapely.geometry import Point

#Librerias de machine learnign
from sklearn import cluster
from sklearn.preprocessing import scale

#Data visualization
import seaborn as sns
sns.set(style="whitegrid")

#### Librerias específicas para la Autocorrelación espacial con Pysal

In [2]:
#For Spatial Correlation
# !pip install pysal
#http://pysal.readthedocs.io/en/latest/
import pysal as ps
from pysal.spreg import ols
from pysal.spreg import ml_error
from pysal.spreg import ml_lag

Recogemos el CSV procedente del Modelo Knn-vecinos con la variable Contaminante categórica codificada

In [3]:
csv_path_geom = '../data/csv/model_csv/df_model_geom_encoding.csv' #Con geometrias para leer con geopandas
shp_path = '../data/shapes/Deaths/Deaths2015.shp'

In [4]:
gdf = gpd.read_file(shp_path)
df = pd.read_csv(csv_path_geom, sep=';', encoding = 'utf-8', compression='gzip', index_col=False)

In [5]:
df.drop(['Unnamed: 0'], axis=1, inplace=True) #Borramos columnas innecesarias

In [6]:
pd.options.display.max_columns = None

In [7]:
df.columns

Index(['target', 'Sexo', 'AnioCumplidos', 'TamanioMuniResi',
       'total_anios_Expo_Id', 'Total_Kg_expo', 'Distance', 'geometry_death',
       'Latitud', 'Longitud', 'Habitantes', 'LatitudE', 'LongitudE',
       'Cont_Dióxido de carbono (CO2)', 'Cont_Partículas (PM10)'],
      dtype='object')

Observamos que todas las variables son numéricas pero habrá que escalar estas

In [8]:
df.dtypes

target                             int64
Sexo                               int64
AnioCumplidos                      int64
TamanioMuniResi                    int64
total_anios_Expo_Id                int64
Total_Kg_expo                    float64
Distance                           int64
geometry_death                    object
Latitud                          float64
Longitud                         float64
Habitantes                         int64
LatitudE                         float64
LongitudE                        float64
Cont_Dióxido de carbono (CO2)      int64
Cont_Partículas (PM10)             int64
dtype: object

#### Pre-processing: Feature Scaling 

In [9]:
from sklearn.preprocessing import StandardScaler

#Standarization de columnas importantes
columnas = ['AnioCumplidos', 'TamanioMuniResi', 'total_anios_Expo_Id', 'Total_Kg_expo']
scaler = StandardScaler()
df[columnas] = scaler.fit_transform(df[columnas]) #Solo columnas de interés escaladas

In [10]:
df.head()

,target,Sexo,AnioCumplidos,TamanioMuniResi,total_anios_Expo_Id,Total_Kg_expo,Distance,geometry_death,Latitud,Longitud,Habitantes,LatitudE,LongitudE,Cont_Dióxido de carbono (CO2),Cont_Partículas (PM10)
0,1,1,-0.979792,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,64595,36.181181,-5.385775,1,0
1,0,0,0.420983,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,64595,36.181181,-5.385775,1,0
2,0,0,-0.463717,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,64595,36.181181,-5.385775,1,0
3,0,0,-1.938217,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,64595,36.181181,-5.385775,1,0
4,0,0,1.084508,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,64595,36.181181,-5.385775,1,0


### Matrices de pesos espaciales (Spatial Weights) 

Debido a que el shape es muy pesado para realizar un análisis de regresión espacial primero vamos a realizar un muestreo aleatorio con numpy

In [11]:
rows = np.random.choice(df.index.values, 5000)

In [12]:
df_sam = df.loc[rows]

In [13]:
df_sam.shape

(5000, 15)

In [14]:
df_sam.head()

,target,Sexo,AnioCumplidos,TamanioMuniResi,total_anios_Expo_Id,Total_Kg_expo,Distance,geometry_death,Latitud,Longitud,Habitantes,LatitudE,LongitudE,Cont_Dióxido de carbono (CO2),Cont_Partículas (PM10)
27451,0,0,1.010783,0.051207,-0.435590,4.176582,5045,POINT (-5.661925999999999 43.54526),43.54526,-5.661926,277554,43.564759,-5.718301,1,0
21302,0,1,-0.537442,-1.453772,-0.993422,1.489945,5843,POINT (-1.240312000000001 38.02786),38.02786,-1.240312,21062,37.975275,-1.236854,1,0
49161,0,0,-0.537442,0.803696,1.795739,-0.521643,427,POINT (2.169919000000001 41.38792),41.38792,2.169919,1621537,41.385048,2.173313,0,1
83713,0,1,0.642158,0.051207,-0.993422,-0.522886,1599,POINT (-6.137173000000002 36.68656),36.68656,-6.137173,207532,36.672600,-6.141580,0,1
60351,0,0,0.273533,0.051207,0.680075,-0.522562,6398,POINT (-16.31477999999999 28.48812),28.48812,-16.314780,150661,28.456375,-16.261808,0,1


### k-nearest neighbor weights

Este método es una combinación de umbrales basados en la distancia junto con los pesos del kernel. En nuestro caso el ancho de banda se establece en un valor predeterminado y se fija a través de las observaciones

In [15]:
del df_sam['geometry_death']

En principio establecemos una matriz de pesos espaciales para k vecinos

In [16]:
wknn = ps.weights.KNN(df_sam, k = 30)

In [17]:
wknn.s0

150000.0

In [18]:
wknn.n

5000

Estandarizamos rows

In [19]:
wknn.transform = 'r'

#### Realizamos una regresión lineal por MCO (Nonspatial regression)

In [20]:
df_sam.columns

Index(['target', 'Sexo', 'AnioCumplidos', 'TamanioMuniResi',
       'total_anios_Expo_Id', 'Total_Kg_expo', 'Distance', 'Latitud',
       'Longitud', 'Habitantes', 'LatitudE', 'LongitudE',
       'Cont_Dióxido de carbono (CO2)', 'Cont_Partículas (PM10)'],
      dtype='object')

In [21]:
np.random.seed(12345)
y = np.array(df_sam['target'])
y.shape = (len(y),1)
X= []
X.append(df_sam['Sexo'])
X.append(df_sam['AnioCumplidos'])
X.append(df_sam['total_anios_Expo_Id'])
X.append(df_sam['Total_Kg_expo'])

X = np.array(X).T

In [22]:
ls = ols.OLS(y, X, name_y = 'Causa fallecimiento CI10-34', name_x = ['Sexo','Años','Total años Expo','Kgs Cont.expo'], name_ds = 'Deaths')
print(ls.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      Deaths
Weights matrix      :        None
Dependent Variable  :Causa fallecimiento CI10-34                Number of Observations:        5000
Mean dependent var  :      0.1494                Number of Variables   :           5
S.D. dependent var  :      0.3565                Degrees of Freedom    :        4995
R-squared           :      0.0516
Adjusted R-squared  :      0.0508
Sum squared residual:     602.620                F-statistic           :     67.9240
Sigma-square        :       0.121                Prob(F-statistic)     :   4.613e-56
S.E. of regression  :       0.347                Log likelihood        :   -1804.925
Sigma-square ML     :       0.121                Akaike info criterion :    3619.850
S.E of regression ML:      0.3472                Schwarz criterion     :    3652.436

--------------------------------------------------------------

In [23]:
lst = df_sam
lst.index = pd.RangeIndex(len(lst.index))
x = ['target', 'Sexo', 'AnioCumplidos', 'TamanioMuniResi',
       'total_anios_Expo_Id', 'Total_Kg_expo', 'Distance', 'geometry_death',
       'Latitud', 'Longitud', 'LatitudE', 'LongitudE',
       'Cont_Dióxido de carbono (CO2)', 'Cont_Partículas (PM10)']

Con la matriz de pesos espaciales vamos a realizar una regressión espacial para ver como afecta la distancia

In [24]:
mi = ps.Moran(ls.u, wknn, two_tailed=False)
print('Observed I:', mi.I, '\nExpected I:', mi.EI, '\n   p-value:', mi.p_norm)

Observed I: 0.14856297727045417 
Expected I: -0.00020004000800160032 
   p-value: 0.0


Utilizamos la función de spatial regression model para cuantificar la no-independencia espacial

In [25]:
np.random.seed(12345)
ye = np.array(df_sam['target'])
ye.shape = (len(ye),1)
Xe= []
Xe.append(df_sam['Distance'])
Xe.append(df_sam['AnioCumplidos'])
Xe.append(df_sam['Sexo'])
Xe.append(df_sam['total_anios_Expo_Id'])
Xe.append(df_sam['Total_Kg_expo'])

Xe = np.array(Xe).T

In [26]:
spat_err = ml_error.ML_Error(ye, Xe, wknn, 
                             name_y='Causa fallecimiento CI10-34', name_x=['Distance','AnioCumplidos','Sexo','Años expo','Cantidad kg expo'], 
                             name_w='deaths wknn', name_ds='Dataframe Deaths')
print(spat_err.summary)

C:\Users\julia\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:643: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :Dataframe Deaths
Weights matrix      : deaths wknn
Dependent Variable  :Causa fallecimiento CI10-34                Number of Observations:        5000
Mean dependent var  :      0.1494                Number of Variables   :           6
S.D. dependent var  :      0.3565                Degrees of Freedom    :        4994
Pseudo R-squared    :      0.0515
Sigma-square ML     :       0.100                Log likelihood        :   -1429.550
S.E of regression   :       0.316                Akaike info criterion :    2871.099
                                                 Schwarz criterion     :    2910.203

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
----------------------------------------

## Modelos de regresión espacial

### Global Autocorrelation

#### Para ver la no independencia espacial, primero utilizamos la función I de Moran desde un punto de vista global

Los valores de I pueden oscilar entre -1 (indicando dispersión perfecta) a 1 (correlación perfecta). Un valor de cero indica un patrón espacial aleatorio.

In [27]:
# Pasamos los weights matrix a la función pysal.Moran, a través de nuestro modelo residual (ls.u)
mi = ps.Moran(ls.u, wknn, two_tailed=False)
print('Observed I:', mi.I, '\nExpected I:', mi.EI, '\n   p-value:', mi.p_norm)

Observed I: 0.14856297727045417 
Expected I: -0.00020004000800160032 
   p-value: 0.0


#### Utilizamos otro estadístico la Geary’s C, que es más sensible a la autocorrelación espacial local.

In [28]:
np.random.seed(12345)
gc = ps.Geary(y, wknn)
print('Observed C:' ,gc.C, '\nExpected C:' ,gc.EC, '\nZ-norm:', gc.z_norm, '\n  p-value:', gc.p_norm)

Observed C: 0.7575098313677547 
Expected C: 1.0 
Z-norm: -49.5131963369163 
  p-value: 0.0


### Local Autocorrelation: Local Indicators of Spatial Association (LISAs) 

Vamos a utilizar un método de autocorrelación local quizá más real para nuestro estudio, utilizamos la I de Moran

In [29]:
lm = ps.Moran_Local(y,wknn)
lm.n

5000

El siguiente método arroja unos pseudo p-values para cada LISA

In [30]:
lm.p_sim

array([0.06 , 0.153, 0.005, ..., 0.151, 0.008, 0.314])

Identificamos los valores LISA por debajo de 0.05 significantes y los indexamos

In [31]:
sig = lm.p_sim < 0.05
print(lm.p_sim[sig])

[0.005 0.03  0.008 ... 0.009 0.015 0.008]


Utilizamos el atributo q para localizar en que cuadrante del Scatter de Moran estan ubicados

In [32]:
lm.q[sig]

array([3, 2, 3, ..., 3, 3, 3])

Vamos a utilizar las variables Sexo y habitantes del municipio basadas en la población con diferentes tamaños para tener en cuenta las diferencias entre la población y estimar el Is de Moran local

In [35]:
b = np.array(df_sam['Sexo'])
e = np.array(df_sam['Habitantes'])

np.random.seed(12345)
lm = ps.esda.moran.Moran_Local_Rate(e, b, wknn)
lm.Is[:]

C:\Users\julia\Anaconda3\lib\site-packages\pysal\esda\smoothing.py:549: RuntimeWarning: divide by zero encountered in true_divide
  y = e * 1.0 / b
C:\Users\julia\Anaconda3\lib\site-packages\pysal\esda\smoothing.py:552: RuntimeWarning: invalid value encountered in multiply
  s2 = sum(b * ((y - ebi_b) ** 2)) / b_sum
C:\Users\julia\Anaconda3\lib\site-packages\pysal\esda\smoothing.py:554: RuntimeWarning: divide by zero encountered in true_divide
  ebi_v_raw = ebi_a + ebi_b / b
C:\Users\julia\Anaconda3\lib\site-packages\pysal\esda\smoothing.py:555: RuntimeWarning: invalid value encountered in less
  ebi_v = np.where(ebi_v_raw < 0, ebi_b / b, ebi_v_raw)
C:\Users\julia\Anaconda3\lib\site-packages\pysal\esda\smoothing.py:555: RuntimeWarning: divide by zero encountered in true_divide
  ebi_v = np.where(ebi_v_raw < 0, ebi_b / b, ebi_v_raw)
C:\Users\julia\Anaconda3\lib\site-packages\pysal\esda\moran.py:892: RuntimeWarning: invalid value encountered in greater
  zp = self.z > 0
C:\Users\julia\Ana

array([nan, nan, nan, ..., nan, nan, nan])

In [36]:
sig = lm.p_sim < 0.05
lm.p_sim[sig]

array([0.001, 0.001, 0.001, ..., 0.001, 0.001, 0.001])